# Logistic regression

- Newton's model

In [233]:
import numpy as np

raw_X = np.loadtxt(path('data/logistic_x.txt')) # m x n
raw_y = np.loadtxt(path('data/logistic_y.txt')) # 1 x m

y = np.array([1 if v == 1 else 0 for v in raw_y])

x_0 = np.ones(X.shape[0]).reshape(-1, 1) # (m,) => (m, 1)
X = np.concatenate((x_0, raw_X), axis=1) # m, n+1

theta = np.zeros(X.shape[1])

print(X[:5])
print(y)
print(theta)

[[ 1.          1.3432504  -1.3311479 ]
 [ 1.          1.8205529  -0.6346681 ]
 [ 1.          0.98632067 -1.8885762 ]
 [ 1.          1.9443734  -1.635452  ]
 [ 1.          0.97673352 -1.3533151 ]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0. 0. 0.]


## Newton's method

Update rule: $\theta := \theta - H^{-1} \nabla_{\theta} l(\theta)$

Partial derivative vector and Hessian:
$$
\begin{aligned}
\nabla_{\theta} \ell (\theta)_{j} &= \sum_{i = 1}^m (y^{(i)} - h_{\theta}(x^{(i)}))x_j^{(i)} \\
\\
H_{kj} &= \frac{\partial^2 \ell(\theta)}{\partial \theta_k \partial \theta_j} \\
&= \sum_{i = 1}^m x_j^{(i)} x_k^{(i)} g(\theta^T x^{(i)}) (1 - g(\theta^T x^{(i)}))
\end{aligned}
$$

In [259]:
def sigmoid(z):
    '''vectorized sigmoid'''
    return 1 / (1 + np.exp(-z))

def hypothesis(theta, X):
    '''vectorized hypothesis, X = design matrix'''
    return sigmoid(X @ theta) # result is length m vector

def partials(theta, X, y):
    '''vectorized partial derivative'''
    h = hypothesis(theta, X)
    residuals = (y - h)
    # jth index of this vector = sum over all training: res * jth feature
    return residuals @ X # result is length n vector

def hessian(theta, X, y):
    prod = hypothesis(theta, X) * (1 - hypothesis(theta, X)) # m-vector
    D = np.diag(prod) # construct diagonal matrix of sigmoid products
    return -X.T @ D @ X # error: was missing the - sign!! spent so long on this

# this cost only works for for y = {-1, 1}
# def cost(theta, X, y):
#     m = X.shape[0]
#     return np.sum(np.log(1 + np.exp( (-y - (X @ theta) ))) / m

def likelihood(theta, X, y):
    class_1 = y @ np.log(hypothesis(theta, X))
    class_0 = (1 - y) @ np.log(1 - hypothesis(theta, X))
    
    return np.sum(class_1 + class_0)
    
    

In [269]:
print(partials(theta, X, y))
print(hessian(theta, X, y))
print(hypothesis(theta, X))

[-6.99440506e-15  2.41091525e-15  1.54579547e-15]
[[  -9.90970038  -38.43213702    2.56818668]
 [ -38.43213702 -182.17193298   17.25050443]
 [   2.56818668   17.25050443  -18.31160651]]
[0.04073121 0.12131754 0.01656349 0.04484215 0.03035866 0.02828366
 0.02905196 0.01920403 0.00827117 0.01096214 0.01366143 0.01057479
 0.00603423 0.01592071 0.02392364 0.0291358  0.10871429 0.11096994
 0.05064028 0.06006143 0.06707658 0.14272094 0.10060626 0.06899812
 0.10186142 0.30013094 0.28640268 0.41153633 0.2928778  0.1408945
 0.06949077 0.09748447 0.08180517 0.04364637 0.02554666 0.75710254
 0.90994524 0.92371256 0.1579067  0.03555153 0.1639652  0.27823047
 0.32164549 0.36227599 0.51603559 0.45973066 0.35379083 0.25140471
 0.91996552 0.33586698 0.93876111 0.95140608 0.89844556 0.90410634
 0.82525191 0.82579046 0.84156756 0.80968021 0.91269446 0.84308645
 0.90957544 0.94732637 0.89031988 0.98679538 0.99405303 0.99324704
 0.98469801 0.99538512 0.87365755 0.82566227 0.92383621 0.87762173
 0.89000912

In [272]:
def newton(theta, X, y, threshold = 0.001, max_iter = 15):

    history = []
    delta = np.Inf
    
    while delta >= threshold and len(history) <= max_iter:
        args = (theta, X, y)
        theta -= np.linalg.pinv(hessian(*args)) @ partials(*args)
        l = likelihood(*args)
        delta = l - history[-1] if len(history) >= 1 else np.Inf
        history.append(l)

    return theta

In [273]:
theta = np.zeros(X.shape[1])
theta = newton(theta, X, y)
print(theta)

[-2.62050954  0.76037096  1.17194549]
